# Demo
> **The purpose of this notebook is to demostrate**
> 1. How to conduct a feature space decomposition
> 2. How to visualize a decomposition
> 3. Qualitatively validate if the algorithm performs as expected

### Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification

from decomposition.var_clus import VarClus

### Demo 1: Instantiate the VarClus class

> **The parameters used in the constructor are**  

> **`n_split`**: Number of sub-clusters that every time a cluster is split into. Default 2  
> **`max_eigenvalue`**: Eigenvalue threshold below which the decomposition will be stopped. Please note, the dataframe will be scaled during the process so each of the features will have variance == 1. Default 1   
> **`max_tries`**: Number of max tries before the algorithm gives up. Default 3

In [2]:
demo1 = VarClus()

# Larger max_eigenvalue usually results in bigger and fewer child clusters
demo1 = VarClus(max_eigenvalue=5)

### Demo 2: Test on an arbitrary dataset

> Let's create some simple dataset to play with  
> We can leverage make_classification to make the dataset

In [23]:
n_features = 25
n_rows = 1e4

raw_df, _ = make_classification(n_samples=int(n_rows), 
                                n_features=n_features, 
                                n_informative=n_features,
                                n_redundant=0)

columns = ['feature_{}'.format(i) for i in range(n_features)]
demo2_df = pd.DataFrame(raw_df, columns=columns)

In [24]:
demo2_df.shape

(10000, 25)

In [27]:
# Supposedly, with max_eigenvalue=1, we should have 4 clusters
demo2 = VarClus(max_eigenvalue=1)
demo2.decompose(demo2_df)

decomposing cluster cluster-0
phase #1: NCS
phase #2: Search
assessing feature feature_0
current EV is 3.853570136359519, new EV is 3.8768876832086407
Feature feature_0 was re-assigned
child_clusters[i] has 12 features and child_clusters[j] has 13 features
assessing feature feature_12
current EV is 3.8768876832086407, new EV is 3.8807093958661545
Feature feature_12 was re-assigned
child_clusters[i] has 11 features and child_clusters[j] has 14 features
assessing feature feature_13
current EV is 3.8807093958661545, new EV is 3.8167121922094784
assessing feature feature_14
current EV is 3.8807093958661545, new EV is 3.833488764533862
assessing feature feature_15
current EV is 3.8807093958661545, new EV is 3.8566614293644523
Number of max tries has been reached. Returning current result...
decomposing cluster cluster-0-0
phase #1: NCS
phase #2: Search
assessing feature feature_14
current EV is 3.078664905854854, new EV is 3.0312034789013116
assessing feature feature_15
current EV is 3.0786

decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0

phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0-0
phase #1: NCS
phase #2: Search
decomposing cluster cluster-0-1-1-1-1-0-0-0-0-0

KeyboardInterrupt: 

In [32]:
a = demo2.cluster.children[-1].children[-1].children[-1].children[-1].children[-1].children[-1]

In [33]:
a.name

'cluster-0-1-1-1-1-0-0'

In [34]:
a.features

['feature_1', 'feature_12']

In [36]:
a.n_split

1

In [35]:
a.pca.explained_variance_

array([ 1.08358684])

In [28]:
# Let's find out
demo2.print_cluster_structure()

cluster-0
|
|-----cluster-0-0
|     |
|     |-----cluster-0-0-0
|     |     |
|     |     |-----cluster-0-0-0-0
|     |     |     |
|     |     |     |-----cluster-0-0-0-0-0
|     |     |     |     |
|     |     |     |     |-----cluster-0-0-0-0-0-0
|     |     |     |     |     |-----feature_20
|     |     |     |     |     |-----feature_23
|     |     |     |     |     |-----feature_4
|     |     |     |     |     |-----feature_5
|     |     |     |     |
|     |     |     |     |-----cluster-0-0-0-0-0-1
|     |     |     |           |-----feature_7
|     |     |     |
|     |     |     |-----cluster-0-0-0-0-1
|     |     |           |-----feature_14
|     |     |           |-----feature_15
|     |     |
|     |     |-----cluster-0-0-0-1
|     |           |-----feature_6
|     |           |-----feature_8
|     |
|     |-----cluster-0-0-1
|           |-----feature_13
|           |-----feature_19
|
|-----cluster-0-1
      |
      |-----cluster-0-1-0
      |     |
      |     |-----clus